This function computes closed null geodesics of $ S(\mathbf{x}, t)-\mu \mathbf{I} $, where $ S(\mathbf{x}, t) = \begin{pmatrix} S^{11} && S^{12} \\ S^{12} && S^{22} \end{pmatrix} $ is the (symmetric) rate of strain tensor at $ (\mathbf{x}, t) $.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| X | array (Ny, Nx) | X-meshgrid|
| Y | array (Ny, Nx) | Y-meshgrid|
| mu | float | $ \mu $|
| interp_phi_prime | interpolant | $ \dot{\phi} $|
| d_threshold | float | threshold distance between start of closed null-geodesic and maximum allowed first return distance to starting point |
| defined domain | boolean array (Ny, Nx) | meshgrid which denotes the domain where the velocity field is defined|
| interp_DOE | interpolant | $ \sin(2\phi)[S^{22}(\mathbf{x})-S^{11}(\mathbf{x})]+2\cos(2\phi)S^{12}(\mathbf{x}) $ |
| solODE_closed_curve | list | list containing the closed solution curves ($ \mathbf{x}, \phi $) |
| x0mu | list | list containing x-coordinates of initial conditions $ x_0(\mu) $ |
| y0mu | list | list containing y-coordinates of initial conditions $ y_0(\mu) $ |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-3])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

In [2]:
# Import numpy
import numpy as np

# Import math symbols
from math import sqrt, pi

# function computing initial conditions (depending on \mu)
from ipynb.fs.defs.init_level_set import _init_level_set

# find closed curve
from ipynb.fs.defs.closed_curve import closed_curve

# RK4 integrator
from ipynb.fs.defs.RK4_integration import RK4_integration

# tqdm shows progress bar
from tqdm.notebook import tqdm

# import parallel computing package
from joblib import Parallel, delayed

# library for Polygon, Point objects
from shapely.geometry import Polygon, Point

# Import (cubic) RectBivariateSpline from scipy
from scipy.interpolate import RectBivariateSpline as RBS

In [ ]:
def closed_null_geodesics(X, Y, mu, interp_phi_prime, d_threshold, S11, defined_domain, interp_DOE):
    
    # domain where the rate of strain field is defined
    defined_domain = np.isfinite(S11).astype(int)
    
    # compute initial conditions
    x0mu, y0mu, phi0mu = _init_level_set(X, Y, S11, mu)
    
    # define integration domain of dummy variable
    s = [0, 10]
    
    # define resolution of trajectories
    s_eval = np.linspace(s[0], s[1], 500)
    
    ds = s_eval[1]-s_eval[0]
    
    # number of initial conditions
    len_x0mu = len(x0mu)
    
    solODE = np.zeros((len(s_eval), 3, len_x0mu))
    
    # initial conditions
    solODE[0, :, :] = np.array([x0mu, y0mu, phi0mu])
    
    for i in range(len(s_eval)-1):
        
        solODE[i+1, :, :] = RK4_integration(s, solODE[i, :, :], ds, interp_phi_prime)
    
    solODE_closed_curves = []
    
    # iterate over all initial conditions [x0mu, y0mu, phi0mu]
    for j in range(len_x0mu):
        
        # store x, y, phi
        x = solODE[:,0, j]
        y = solODE[:,1, j]
        phi = solODE[:,2, j]
    
        # Check if curve is closed after completing one full cycle and find curve with minimum re-intersection distance.
        x_closed, y_closed, phi_closed = closed_curve(x, y, phi, d_threshold, X, Y, interp_DOE, defined_domain)
        
        # store solutions
        solODE_closed_curves.append([x_closed, y_closed, phi_closed])
    
    return solODE_closed_curves, [x0mu, y0mu]